In [28]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
import scipy.stats as stats

mort = "./2012-2016 CardioVasc Mortality Rates.csv"
edu = "./Education.csv"
econo = "./SAIPESNC.csv"

In [2]:
mortality_file = pd.read_csv(mort)
education_file = pd.read_csv(edu)
socio_econo_file = pd.read_csv(econo)

print(mortality_file.columns)
print()
print(education_file.columns)
print()
print(socio_econo_file.columns)

Index(['Notes', 'State', 'State Code', 'County', 'County Code', 'Deaths',
       'Population', 'Crude Rate', 'Crude Rate Lower 95% Confidence Interval',
       'Crude Rate Upper 95% Confidence Interval',
       'Crude Rate Standard Error'],
      dtype='object')

Index(['FIPS Code', 'State', 'Area name',
       'Less than a high school diploma, 2012-2016',
       'High school diploma only, 2012-2016',
       'Some college or associate's degree, 2012-2016',
       'Bachelor's degree or higher, 2012-2016',
       'Percent of adults with less than a high school diploma, 2012-2016',
       'Percent of adults with a high school diploma only, 2012-2016',
       'Percent of adults completing some college or associate's degree, 2012-2016',
       'Percent of adults with a bachelor's degree or higher, 2012-2016'],
      dtype='object')

Index(['State', 'County ID', 'State / County Name',
       'All Ages SAIPE Poverty Universe', 'All Ages in Poverty Count',
       'All Ages in Poverty Count LB 

In [3]:
education_file = education_file.rename(columns={'FIPS Code':'County ID'})
education_file.head()

,County ID,State,Area name,"Less than a high school diploma, 2012-2016","High school diploma only, 2012-2016","Some college or associate's degree, 2012-2016","Bachelor's degree or higher, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a bachelor's degree or higher, 2012-2016"
0,0,US,United States,"27,818,380","58,820,411","62,242,569","64,767,787",13.0,27.5,29.1,30.3
1,1001,AL,Autauga County,"4,528","12,519","10,451","8,968",12.4,34.3,28.7,24.6
2,1003,AL,Baldwin County,"13,956","40,154","44,486","41,350",10.0,28.7,31.8,29.5
3,1005,AL,Barbour County,"4,824","6,422","4,775","2,366",26.2,34.9,26.0,12.9
4,1007,AL,Bibb County,"3,040","6,586","4,234","1,890",19.3,41.8,26.9,12.0


In [4]:
new_edu_columns = ['County ID', 'State', "County Name", '< High School', 'High School', 'Associate',
                  'Bachelors <', 'Perc < High School', 'Perc High School', 'Perc Associate',
                  'Perc Bachelors <']
education_file.columns = new_edu_columns

In [5]:
edu_cleaned_columns = ['County ID', "County Name", 'Perc < High School', 'Perc High School', 'Perc Associate', 'Perc Bachelors <']
education_clean_df = education_file[edu_cleaned_columns]
education_clean_df['MYS'] = ""
education_clean_df.head()

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS
0,0,United States,13.0,27.5,29.1,30.3,
1,1001,Autauga County,12.4,34.3,28.7,24.6,
2,1003,Baldwin County,10.0,28.7,31.8,29.5,
3,1005,Barbour County,26.2,34.9,26.0,12.9,
4,1007,Bibb County,19.3,41.8,26.9,12.0,


In [6]:
mean_years_schooling = []
for index, row in education_clean_df.iterrows():
    mean_years_schooling.append(((row['Perc < High School']*10) + (row['Perc High School'] * 12) + (row['Perc Associate'] * 14)
                              + (row['Perc Bachelors <'] * 20))/100)
    
education_clean_df['MYS'] = mean_years_schooling
education_clean_df['MYS'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    3143.000000
mean       13.989138
std         0.834010
min        12.010000
10%        13.038400
20%        13.280800
30%        13.502000
40%        13.706000
50%        13.882000
60%        14.070000
70%        14.290800
80%        14.603200
90%        15.104800
max        18.602000
Name: MYS, dtype: float64

In [7]:
education_clean_df['Education Bin'] = ""

education_bins = [12, 13.05, 13.28, 13.502, 13.706, 13.882, 14.07, 14.291, 14.6032, 15.1048, 18.603]
education_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
education_clean_df['Education Bin'] = pd.cut(education_clean_df['MYS'], bins=education_bins, labels=education_bin_labels)
education_clean_df.head()

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,Education Bin
0,0,United States,13.0,27.5,29.1,30.3,14.734,9
1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294,8
2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796,9
3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028,1
4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112,2


In [8]:
# Removing county 15005 from our education dataset as well, since it isn't included in our socio_econo file
education_clean_df = education_clean_df[education_clean_df['County ID'] != 15005]
education_clean_df.reset_index().head()

,index,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,Education Bin
0,0,0,United States,13.0,27.5,29.1,30.3,14.734,9
1,1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294,8
2,2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796,9
3,3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028,1
4,4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112,2


In [9]:
education_clean_df['Education Bin'].value_counts()

1     328
4     316
10    315
3     315
8     314
6     314
5     314
9     313
7     312
2     301
Name: Education Bin, dtype: int64

In [10]:
socio_columns = ['State', 'County ID', 'County Name', '1', '2', '3', '4',' 5', '6','7', 'Median Household Income', '8', '9']
socio_econo_file.columns = socio_columns
socio_econo_file.head()

,State,County ID,County Name,1,2,3,4,5,6,7,Median Household Income,8,9
0,0,0,United States,310764925,47240290,46990033,47490547,15.2,15.1,15.3,54134,54052,54216
1,1,1001,Autauga County (AL),54905,7206,5914,8497,13.1,10.8,15.5,53748,49752,57745
2,1,1003,Baldwin County (AL),197091,25850,21888,29811,13.1,11.1,15.1,50976,48035,53917
3,1,1005,Barbour County (AL),23311,6731,5511,7952,28.9,23.6,34.1,32111,29316,34907
4,1,1007,Bibb County (AL),20316,4268,3359,5177,21.0,16.5,25.5,40113,36439,43787


In [11]:
# Change county name to just have county name, not state name, too.
socio_cleaned_columns = ['County ID', 'County Name', 'Median Household Income']
socio_econo_clean_df = socio_econo_file[socio_cleaned_columns]
socio_econo_clean_df.head()

,County ID,County Name,Median Household Income
0,0,United States,54134
1,1001,Autauga County (AL),53748
2,1003,Baldwin County (AL),50976
3,1005,Barbour County (AL),32111
4,1007,Bibb County (AL),40113


In [12]:
# socio_econo_clean_df['Median Household Income'].describe()
# Need to change this mofo to an integer so  we can get stats on it...
# pd.to_numeric(socio_econo_clean_df['Median Household Income'])
print(socio_econo_clean_df.iloc[549:555, :])
# C.ID 15005 is filled with #DIV/O! errors in all columns in the row, just going to drop this from our dataset, in all csv files.

     County ID           County Name Median Household Income
549      15001    Hawaii County (HI)                   51582
550      15003  Honolulu County (HI)                   74794
551      15005   Kalawao County (HI)                 #DIV/0!
552      15007     Kauai County (HI)                   60725
553      15009      Maui County (HI)                   64971
554      16001       Ada County (ID)                   57133


In [13]:
# Dropped the fucked row and reset index
socio_econo_clean_df = socio_econo_clean_df[socio_econo_clean_df['County ID'] != 15005]
socio_econo_clean_df.reset_index().head()

,index,County ID,County Name,Median Household Income
0,0,0,United States,54134
1,1,1001,Autauga County (AL),53748
2,2,1003,Baldwin County (AL),50976
3,3,1005,Barbour County (AL),32111
4,4,1007,Bibb County (AL),40113


In [14]:
print(socio_econo_clean_df.iloc[549:555, :])

     County ID           County Name Median Household Income
549      15001    Hawaii County (HI)                   51582
550      15003  Honolulu County (HI)                   74794
552      15007     Kauai County (HI)                   60725
553      15009      Maui County (HI)                   64971
554      16001       Ada County (ID)                   57133
555      16003     Adams County (ID)                   38973


In [15]:
# Now, we can convert Income column to integer
socio_econo_clean_df['Median Household Income'] = pd.to_numeric(socio_econo_clean_df['Median Household Income'], errors='coerce')

In [16]:
socio_econo_clean_df['Median Household Income'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

count      3146.000000
mean      47211.750795
std       11968.915769
min       22761.000000
10%       34365.500000
20%       37776.000000
30%       40560.500000
40%       42962.000000
50%       45397.000000
60%       48096.000000
70%       50937.500000
80%       54659.000000
90%       61463.000000
max      123953.000000
Name: Median Household Income, dtype: float64

In [17]:
# We will create the same bins based on these percentile ranges and see if they match up with our education bins
socio_econo_clean_df['Income Bin'] = ""

income_bins = socio_econo_clean_df['Median Household Income'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 
                                                                                    0.8, 0.9])[['min', '10%', '20%', '30%', 
                                                                                                '40%', '50%', '60%', '70%', 
                                                                                                '80%', '90%', 'max']]
income_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
socio_econo_clean_df['Income Bin'] = pd.cut(socio_econo_clean_df['Median Household Income'], 
                                            bins=income_bins, labels=income_bin_labels)
socio_econo_clean_df.head()

,County ID,County Name,Median Household Income,Income Bin
0,0,United States,54134,8
1,1001,Autauga County (AL),53748,8
2,1003,Baldwin County (AL),50976,8
3,1005,Barbour County (AL),32111,1
4,1007,Bibb County (AL),40113,3


In [19]:
# Now, join the two files on 'County ID'
# We are going to have two 'County Name' columns now, we will compare values to make sure merge happened as expected, then delete one
pd.merge(education_clean_df, socio_econo_clean_df, on='County ID', how='outer', suffixes=('_ED', 'SE'), validate='1:1')
# Throws an error, keys are not unique in our socio_econo file, this is just to show we cannot proceed with our current dataframes
# and that we need to clean it up a little more before merging...  Skip this cell and move on to the next steps.

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

In [20]:
socio_econo_clean_df['County ID'].value_counts() > 1
# This shows we have duplicates...  Need to drop one of them.

2195      True
47103    False
28019    False
28033    False
28031    False
28029    False
28027    False
48415    False
28025    False
28023    False
5147     False
28021    False
47003    False
28035    False
28017    False
31153    False
28015    False
28013    False
28011    False
28009    False
17195    False
28007    False
28005    False
39121    False
56031    False
28037    False
26123    False
28051    False
28065    False
20209    False
         ...  
47009    False
17053    False
17051    False
42043    False
17049    False
17075    False
20121    False
17077    False
17093    False
13011    False
13009    False
48277    False
13007    False
13005    False
13003    False
13001    False
47075    False
17095    False
17091    False
17079    False
17089    False
42105    False
17087    False
13303    False
17085    False
55041    False
17083    False
17183    False
17081    False
0        False
Name: County ID, Length: 3145, dtype: bool

In [21]:
socio_econo_clean_df = socio_econo_clean_df.drop_duplicates(['County ID'], keep='first')

In [22]:
socio_econo_clean_df['County ID'].value_counts(sort=True)
# This shows that all duplicate values have been dropped (sorts from hi to lo, all values are 1)

47103    1
28035    1
28033    1
28031    1
28029    1
28027    1
48415    1
28025    1
28023    1
5147     1
28021    1
28019    1
47003    1
28017    1
31153    1
28015    1
28013    1
28011    1
28009    1
17195    1
28007    1
28005    1
39121    1
56031    1
28037    1
48421    1
28039    1
28067    1
28065    1
20209    1
        ..
47009    1
17053    1
17051    1
42043    1
17049    1
17075    1
20121    1
17077    1
17093    1
13011    1
13009    1
48277    1
13007    1
13005    1
13003    1
13001    1
47075    1
17095    1
17091    1
17079    1
17089    1
42105    1
17087    1
13303    1
17085    1
55041    1
17083    1
17183    1
17081    1
0        1
Name: County ID, Length: 3145, dtype: int64

In [23]:
print(education_clean_df.shape)
print(socio_econo_clean_df.shape)
# This shows that there are a few extra rows in our education dataset that will need to be addressed, as they will be occupied 
# by empty or NaN values

(3152, 8)
(3145, 4)


In [24]:
# Now, we should be able to merge everything...
education_income_df = pd.merge(education_clean_df, socio_econo_clean_df, on='County ID', how='outer', 
                               suffixes=('_ED', 'SE'), validate='1:1')

education_income_df.head()

,County ID,County Name_ED,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,Education Bin,County NameSE,Median Household Income,Income Bin
0,0,United States,13.0,27.5,29.1,30.3,14.734,9,United States,54134.0,8
1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294,8,Autauga County (AL),53748.0,8
2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796,9,Baldwin County (AL),50976.0,8
3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028,1,Barbour County (AL),32111.0,1
4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112,2,Bibb County (AL),40113.0,3


In [25]:
education_income_df.columns

Index(['County ID', 'County Name_ED', 'Perc < High School', 'Perc High School',
       'Perc Associate', 'Perc Bachelors <', 'MYS', 'Education Bin',
       'County NameSE', 'Median Household Income', 'Income Bin'],
      dtype='object')

In [26]:
education_income_clean_df = education_income_df[['County ID', 'Perc < High School', 'Perc High School',
       'Perc Associate', 'Perc Bachelors <', 'MYS', 'Education Bin',
       'County NameSE', 'Median Household Income', 'Income Bin']]
education_income_clean_df.head()

,County ID,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,Education Bin,County NameSE,Median Household Income,Income Bin
0,0,13.0,27.5,29.1,30.3,14.734,9,United States,54134.0,8
1,1001,12.4,34.3,28.7,24.6,14.294,8,Autauga County (AL),53748.0,8
2,1003,10.0,28.7,31.8,29.5,14.796,9,Baldwin County (AL),50976.0,8
3,1005,26.2,34.9,26.0,12.9,13.028,1,Barbour County (AL),32111.0,1
4,1007,19.3,41.8,26.9,12.0,13.112,2,Bibb County (AL),40113.0,3


In [27]:
# Move county name to front and rename
education_income_clean_df = education_income_clean_df[['County NameSE', 'County ID', 'Perc < High School', 'Perc High School',
       'Perc Associate', 'Perc Bachelors <', 'MYS', 'Education Bin',
       'Median Household Income', 'Income Bin']]
education_income_clean_df = education_income_clean_df.rename(columns={'County NameSE' : 'County Name (State)', 
                                                                      'Perc < High School' : '% < High School', 
                                                                      'Perc High School' : '% High School' ,
                                                                      'Perc Associate' : '% Associate', 
                                                                      'Perc Bachelors <' : '% Bachelors <'})
education_income_clean_df.head()

,County Name (State),County ID,% < High School,% High School,% Associate,% Bachelors <,MYS,Education Bin,Median Household Income,Income Bin
0,United States,0,13.0,27.5,29.1,30.3,14.734,9,54134.0,8
1,Autauga County (AL),1001,12.4,34.3,28.7,24.6,14.294,8,53748.0,8
2,Baldwin County (AL),1003,10.0,28.7,31.8,29.5,14.796,9,50976.0,8
3,Barbour County (AL),1005,26.2,34.9,26.0,12.9,13.028,1,32111.0,1
4,Bibb County (AL),1007,19.3,41.8,26.9,12.0,13.112,2,40113.0,3


In [31]:
education_income_clean_df.shape
# This means there are more rows than either of the two original dataframes had initially...  drop those NaN's.

(3154, 10)

In [35]:
education_income_clean_df = education_income_clean_df.dropna()
education_income_clean_df.shape

(3141, 10)

In [38]:
education_income_clean_df['County ID'].count()

3141

In [40]:
# Chi square test comparing edu against income
critical_value = stats.chi2.ppf(q=0.99, df=3140)
critical_value

3327.2907255001487

In [41]:
stats.chisquare(education_income_clean_df['Education Bin'], education_income_clean_df['Income Bin'])

TypeError: unsupported operand type(s) for -: 'str' and 'str'